# <center> Тестовое задание для вакансии “Backend разработчик (Python, Junior)”

## <center> Загружаем данные

In [1]:
import json
import pandas as pd

In [2]:
file_path = 'trial_task.json'

with open(file_path, 'r') as json_file:
    data = json.load(json_file)


product_records = []

for order in data:
    order_id = order['order_id']
    warehouse_name = order['warehouse_name']
    highway_cost = order['highway_cost']
    for product_data in order['products']:
        product = product_data['product']
        price = product_data['price']
        quantity = product_data['quantity']

        product_record = {
            'order_id': order_id,
            'warehouse_name': warehouse_name,
            'highway_cost': highway_cost,
            'product': product,
            'price': price,
            'quantity': quantity
        }

        product_records.append(product_record)

df = pd.DataFrame(product_records)

df

,order_id,warehouse_name,highway_cost,product,price,quantity
0,11973,Мордор,-70,ломтик июльского неба,450,1
1,11973,Мордор,-70,билет в Израиль,1000,3
2,11973,Мордор,-70,статуэтка Ленина,200,3
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1
4,85794,отель Лето,-50,зеленая пластинка,10,2
...,...,...,...,...,...,...
192,79293,отель Лето,-75,автограф Стаса Барецкого,600,1
193,79293,отель Лето,-75,ломтик июльского неба,450,1
194,2930,Мордор,-30,плюмбус,250,2
195,2930,Мордор,-30,плюмбус,250,1


## <center> Задачи

### 1. Найти тариф стоимости доставки для каждого склада

Сначала находим общее количество единиц товара для каждого заказа:

In [3]:
qty_per_order = df.groupby('order_id')['quantity'].sum().reset_index()
qty_per_order.rename(columns={'quantity': 'order_total_qty'}, inplace=True)

qty_per_order

,order_id,order_total_qty
0,124,3
1,1391,2
2,2091,4
3,2108,2
4,2558,5
...,...,...
95,98100,2
96,98423,2
97,99220,5
98,99246,3


Теперь присоединяем эти количества к основной таблице и рассчитываем тариф для каждого товара:

In [4]:
df = df.merge(qty_per_order, on='order_id')

df['tariff'] = df['highway_cost'] // df['order_total_qty']

df

,order_id,warehouse_name,highway_cost,product,price,quantity,order_total_qty,tariff
0,11973,Мордор,-70,ломтик июльского неба,450,1,7,-10
1,11973,Мордор,-70,билет в Израиль,1000,3,7,-10
2,11973,Мордор,-70,статуэтка Ленина,200,3,7,-10
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1,1,-15
4,85794,отель Лето,-50,зеленая пластинка,10,2,2,-25
...,...,...,...,...,...,...,...,...
192,79293,отель Лето,-75,автограф Стаса Барецкого,600,1,3,-25
193,79293,отель Лето,-75,ломтик июльского неба,450,1,3,-25
194,2930,Мордор,-30,плюмбус,250,2,3,-10
195,2930,Мордор,-30,плюмбус,250,1,3,-10


Теперь мы можем вывести тарифы для каждого склада:

In [5]:
df.groupby('warehouse_name')['tariff'].first().reset_index()

,warehouse_name,tariff
0,Мордор,-10
1,гиперборея,-20
2,остров невезения,-5
3,отель Лето,-25
4,хутор близ Диканьки,-15


### 2. Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit')

Добавляем в основную таблицу стоблцы с доходом, расходом и прибылью:

In [6]:
df['income'] = df['price'] * df['quantity']
df['expenses'] = df['tariff'] * df['quantity']
df['profit'] = df['income'] + df['expenses']

df.head()

,order_id,warehouse_name,highway_cost,product,price,quantity,order_total_qty,tariff,income,expenses,profit
0,11973,Мордор,-70,ломтик июльского неба,450,1,7,-10,450,-10,440
1,11973,Мордор,-70,билет в Израиль,1000,3,7,-10,3000,-30,2970
2,11973,Мордор,-70,статуэтка Ленина,200,3,7,-10,600,-30,570
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1,1,-15,1000,-15,985
4,85794,отель Лето,-50,зеленая пластинка,10,2,2,-25,20,-50,-30


Группируем по продукту:

In [7]:
products = df.groupby('product')[['quantity', 'income', 'expenses', 'profit']].sum().reset_index()
products

,product,quantity,income,expenses,profit
0,автограф Стаса Барецкого,48,28800,-820,27980
1,билет в Израиль,58,58000,-1050,56950
2,зеленая пластинка,61,610,-920,-310
3,ломтик июльского неба,42,18900,-670,18230
4,плюмбус,65,16250,-940,15310
5,подписка на suppi-блог,33,4950,-545,4405
6,статуэтка Ленина,68,13600,-935,12665


### 3. Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

Находим суммарную прибыль каждого заказа:

In [8]:
profit_per_order = df.groupby('order_id')['profit'].sum().reset_index()
profit_per_order.rename(columns={'profit': 'order_profit'}, inplace=True)
profit_per_order

,order_id,order_profit
0,124,705
1,1391,490
2,2091,1300
3,2108,200
4,2558,355
...,...,...
95,98100,1590
96,98423,1170
97,99220,1075
98,99246,1755


Находим среднюю прибыль заказов:

In [9]:
profit_per_order['order_profit'].mean()

1352.3

### 4. Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

Находим суммарную прибыль по каждому заказу:

In [10]:
warehouse_profit = df.groupby(['warehouse_name'])['profit'].sum().reset_index()
warehouse_profit.rename(columns={'profit': 'warehouse_profit'}, inplace=True) 

warehouse_profit

,warehouse_name,warehouse_profit
0,Мордор,17050
1,гиперборея,38280
2,остров невезения,15610
3,отель Лето,27050
4,хутор близ Диканьки,37240


Группируем основную таблицу по складу и продукту. Для расчёта процента прибыли продукта добавляем столбец с суммарной прибылью по каждому складу, а после расчёта - удаляем этот столбец.

In [11]:
product_profit_percent = df.groupby(
    ['warehouse_name', 'product']
)[['quantity', 'profit']].sum().reset_index().merge(
    warehouse_profit, on='warehouse_name'
)

product_profit_percent['percent_profit_product_of_warehouse'] = (
    product_profit_percent['profit'] / product_profit_percent['warehouse_profit'] * 100
)
product_profit_percent.drop(columns='warehouse_profit', inplace=True)

product_profit_percent

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,4,2360,13.841642
1,Мордор,билет в Израиль,9,8910,52.258065
2,Мордор,зеленая пластинка,11,0,0.000000
3,Мордор,ломтик июльского неба,3,1320,7.741935
4,Мордор,плюмбус,6,1440,8.445748
5,Мордор,подписка на suppi-блог,8,1120,6.568915
6,Мордор,статуэтка Ленина,10,1900,11.143695
7,гиперборея,автограф Стаса Барецкого,12,6960,18.181818
8,гиперборея,билет в Израиль,21,20580,53.761755
9,гиперборея,зеленая пластинка,10,-100,-0.261233


### 5. Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться 'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

Группируем предыдущую таблицу по складу, внутри группы сортируем по убыванию и добавляем накопленный процент:

In [12]:
product_profit_cum = product_profit_percent.groupby(
    'warehouse_name'
).apply(
    lambda x: x.sort_values('percent_profit_product_of_warehouse', ascending=False)
).reset_index(drop=True)

product_profit_cum['accumulated_percent_profit_product_of_warehouse'] = (
    product_profit_cum.groupby('warehouse_name')['percent_profit_product_of_warehouse'].cumsum()
)

product_profit_cum

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
0,Мордор,билет в Израиль,9,8910,52.258065,52.258065
1,Мордор,автограф Стаса Барецкого,4,2360,13.841642,66.099707
2,Мордор,статуэтка Ленина,10,1900,11.143695,77.243402
3,Мордор,плюмбус,6,1440,8.445748,85.689150
4,Мордор,ломтик июльского неба,3,1320,7.741935,93.431085
5,Мордор,подписка на suppi-блог,8,1120,6.568915,100.000000
6,Мордор,зеленая пластинка,11,0,0.000000,100.000000
7,гиперборея,билет в Израиль,21,20580,53.761755,53.761755
8,гиперборея,автограф Стаса Барецкого,12,6960,18.181818,71.943574
9,гиперборея,ломтик июльского неба,13,5590,14.602926,86.546499


### 6. Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). Если значение накопленного процента меньше или равно 70, то категория A. Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

Для присвоения категории создаём функцию, которую затем используем для создания столбца с категорией:

In [13]:
def category(cum_percentage):
    if cum_percentage <= 70:
        return 'А'
    elif cum_percentage <= 90:
        return 'Б'
    else:
        return 'С'
    

product_profit_cum['category'] = (
    product_profit_cum['accumulated_percent_profit_product_of_warehouse'].apply(category)
)
product_profit_cum

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
0,Мордор,билет в Израиль,9,8910,52.258065,52.258065,А
1,Мордор,автограф Стаса Барецкого,4,2360,13.841642,66.099707,А
2,Мордор,статуэтка Ленина,10,1900,11.143695,77.243402,Б
3,Мордор,плюмбус,6,1440,8.445748,85.689150,Б
4,Мордор,ломтик июльского неба,3,1320,7.741935,93.431085,С
5,Мордор,подписка на suppi-блог,8,1120,6.568915,100.000000,С
6,Мордор,зеленая пластинка,11,0,0.000000,100.000000,С
7,гиперборея,билет в Израиль,21,20580,53.761755,53.761755,А
8,гиперборея,автограф Стаса Барецкого,12,6960,18.181818,71.943574,Б
9,гиперборея,ломтик июльского неба,13,5590,14.602926,86.546499,Б
